# Gravitation constrained to a surface

Consider a point mass $m$ constrained to a surface in the 3-D space. The system is submitted to a uniform constant acceleration $g$ and moves without friction. For simplicity, we assume that the surface is stable by rotation around an axis $\Delta$ parallel to $g$. The referential is taken to be centred on a point $O$ of $\Delta$. Its $z$-axis is taken to be the semi-axis of $\Delta$ in the opposite direction to $g$. The $x$- and $y$-axes are chosen arbitrarily so as to form an orthogonal basis.

The generalised coordinates of the system are the angle $\phi$ ("azimuth") which is the polar coordinate of the projection of the point mass on the $x,y$-plane, and the angle $\theta$ ("zenith") which is the angle of the $z$-axis with the point mass. We have $0\leq\theta\leq\pi$, and any two $\theta,\phi$ identify a single point on the surface at radius $R(\theta)$. The coordinates of the point mass are then given by


\begin{equation*}
x = R\sin\theta\cos\phi
\hspace{2cm}
y = R\sin\theta\sin\phi
\hspace{2cm}
z = -R\cos\theta
\end{equation*}

The only active force is weight with coordinates $(0,0,-mg)$, hence of the form $-\nabla V$ with $V=mgz=-mgR\cos\theta$. Hence the Lagrangian

\begin{eqnarray*}
\mathcal{L} & = & m\left(\frac{1}{2}R^2\dot{\phi}^2\sin^2\theta+\frac{1}{2}(R^2+R'^2)\dot{\theta}^2+gR\cos\theta\right)
\end{eqnarray*}

Introducing the intermediary variable $q=R^2\dot{\phi}\sin^2\theta$ (angular moment), the equations of the dynamics are given by:

\begin{equation*}
\begin{array}{rrcl}
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{\phi}} = \frac{\partial\mathcal{L}}{\partial\phi}\right]
\hspace{1cm} &
\dot{\phi} & = & \frac{q}{R^2\sin^2\theta}
\\
& \dot{q} & = & 0
\\
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{\theta}} = \frac{\partial\mathcal{L}}{\partial\theta}\right]
\hspace{1cm} &
\ddot{\theta} & = & \frac{1}{R^2+R'^2}\left(\frac{q^2}{R^3\sin^3\theta}(R'\sin\theta+R\cos\theta)+g(R'\cos\theta-R\sin\theta)-R'(R+R'')\dot{\theta}^2\right)
\end{array}
\end{equation*}

For a sphere, $R$ is constant and $R'=R''=0$ and the system simplifies to

\begin{equation*}
\begin{array}{rcl}
\ddot{\theta} & = & \frac{q^2\cos\theta}{R^4\sin^3\theta}-\frac{g}{R}\sin\theta\\
\dot{q} & = & 0
\end{array}
\end{equation*}

Documentation: [here](../doc/_build/html/odesimu.html)

In [1]:
#%pylab nbagg
%pylab
# for better display performance

import logging
logger = logging.getLogger()

from functools import partial
from collections import ChainMap
from scipy.integrate import quad
from ipyshow.odesimu.system import System
from ipyshow.odesimu.util import logger_hook

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
#----------------------------------------------------------------------------------------------------
class SphereSlider (System):
#----------------------------------------------------------------------------------------------------

    shadowshape = (5,)

    def __init__(self,R,G):
        from matplotlib.cm import get_cmap
        self.R, self.G = R, G
        def main(t,state,a=G/R):
            theta,dtheta,phi,q = state
            c,s = cos(theta),sin(theta)
            q2,s2 = square(q),square(s)
            ddtheta = q2/s2*c/s-a*s
            dphi = q/s2
            dq = 0
            return array((dtheta,ddtheta,dphi,dq))
        self.main = main
        def jac(t,state,a=G/R):
            theta,dtheta,phi,q = state
            c,s = cos(theta),sin(theta)
            q2,s2 = square(q),square(s)
            v = 2*q/s2*c/s
            return array(((0,1,0,0),(q2/s2*(2-3/s2)-a*c,0,0,v),(-v,0,0,1/s2),(0,0,0,0)))
        self.jacobian = jac
        def fordisplay(state,cmap=get_cmap('rainbow')):
            theta,dtheta,phi,q = state
            r = R*sin(theta)
            live = (r*cos(phi),r*sin(phi))+cmap((1-cos(theta))/2)[:-1]
            return live, live
        self.fordisplay = fordisplay

    Help = '''
        /G [m.sec^-2]: gravitational acceleration
        /R [m]: radius of the sphere
    '''

    def display(self,ax,refsize=50.,**ka):
        from matplotlib.patches import Circle
        R = 1.05*self.R
        ax.set_xlim(-R,R)
        ax.set_ylim(-R,R)
        ax.scatter((0.,),(0.,),c='k',marker='o',s=refsize)
        ax.add_patch(Circle((0,0),self.R,edgecolor='k',facecolor='none'))
        diag_s = ax.scatter((),(),s=refsize,marker='o',c='r')
        tail_l, = ax.plot((),(),'gray')
        tail_s = ax.scatter((),(),s=refsize,marker='o')
        def disp(t,live,tail):
            x,y = live[:2]
            diag_s.set_offsets(((x,y),))
            diag_s.set_color(live[2:])
            tail_l.set_data(tail[:,0],tail[:,1])
            tail_s.set_offsets(tail[:,0:2])
            tail_s.set_color(tail[:,2:])
        return super().display(ax,disp,**ka)

    @staticmethod
    def makestate(theta=0.,wtheta=0,phi=0.,wphi=0.):
        s = sin(theta*pi/180)
        return array((theta,wtheta,phi,wphi*square(s)),float)*pi/180.

    Help += '''
        makestate/theta [deg]: zenith
        makestate/wtheta [deg.sec^-1]: angular speed (zenith)
        makestate/phi [deg]: azimuth
        makestate/wphi [deg.sec^-1]: angular speed (azimuth)
        launch/refsize [pt^2]: size (area) of the blob for display
    '''

    launchdefaults = ChainMap(
        dict(
            taild=1,
            hooks=(partial(logger_hook,logger=logger),),
        ),
        System.launchdefaults
    )

    Help += System.Help

In [3]:
syst = SphereSlider(R=2.,G=9.81); ini = dict(theta=135,wtheta=0,wphi=30)
syst.launch(ini=syst.makestate(**ini))